# 📊 04 - Analyse de Données avec Spark
Ce notebook a pour objectif d'analyser les données déjà transformées, nettoyées et enrichies.


In [1]:
# 0. Stoppe toute session existante
try:
    spark.stop()
except:
    pass

# Recréation SparkSession en local, FS local et driver binding fixe
from pyspark.sql import SparkSession
spark = (
    SparkSession.builder
        .appName("03_Enrichment")
        .master("local[*]")
        .config("spark.hadoop.fs.defaultFS", "file:///")
        .config("spark.driver.host", "127.0.0.1")
        .config("spark.driver.bindAddress", "0.0.0.0")
        .getOrCreate()
)


In [2]:
from pyspark.sql.functions import col, lower, trim, count
import os

# 1. Chargement
path = os.path.abspath(os.path.join(os.getcwd(), "../data/step3_enriched_csv"))
df_enriched = spark.read.option("header", "true").option("sep", ";").csv(path)

In [3]:
# Marques les plus fréquentes
top_brands = df_enriched.groupBy("brands") \
    .count() \
    .orderBy("count", ascending=False) \
    .filter(col("brands").isNotNull() & (col("brands") != "")) \
    .limit(30)

# Sauvegarde pour la visualisation
top_brands.coalesce(1) \
    .write.option("header", "true") \
    .mode("overwrite") \
    .csv("../data/viz_top_brands")

In [4]:
# Pays les plus fréquents
top_countries = df_enriched.groupBy("country") \
    .count() \
    .orderBy("count", ascending=False) \
    .filter(col("country").isNotNull() & (col("country") != "")) \
    .limit(20)

top_countries.coalesce(1) \
    .write.option("header", "true") \
    .mode("overwrite") \
    .csv("../data/viz_top_countries")